In [1]:
from Preprocessing_M import DataExtractor, DataPreprocessing

de = DataExtractor("D:/hka-aqm-m", "C:/Users/danie/building_M_data")

df = de.create_df()

No .dat files found in C:/Users/danie/building_M_data. 
 Trying to extract files from the original directory D:/hka-aqm-m
Read data successfully.
Data contains 979166 data points and 18 columns.


In [2]:
dp = DataPreprocessing()

df_preprocessed = dp.preprocess_df(df, rolling_window = "7d", sample_time = "1d")

In [3]:
df_preprocessed.to_parquet("C:/Users/danie/hka_M_preprocessed.parquet")

In [4]:
import pandas as pd
df_preprocessed = pd.read_parquet("C:/Users/danie/hka_M_preprocessed.parquet")

In [6]:
from Feature_Engineering import *
fe = FeatureEngineering(df_preprocessed, label = "tmp", categorical_features = ["season", "room_number", "dayofweek", "color"])

In [25]:
fe.X_train.columns

Index(['year', 'hour', 'season_fall', 'season_spring', 'season_summer',
       'season_winter', 'room_number_m001', 'room_number_m002',
       'room_number_m003', 'room_number_m004', 'room_number_m101',
       'room_number_m102', 'room_number_m103', 'room_number_m104',
       'room_number_m105', 'room_number_m113', 'room_number_m201',
       'room_number_m202', 'room_number_m206', 'room_number_m207',
       'room_number_m209', 'room_number_m216', 'room_number_m301',
       'room_number_m302', 'room_number_m303', 'room_number_m304',
       'room_number_m306', 'room_number_m310', 'room_number_m700',
       'room_number_mu04', 'room_number_mu06', 'room_number_mu10',
       'room_number_mu11', 'dayofweek_0', 'dayofweek_1', 'dayofweek_2',
       'dayofweek_3', 'dayofweek_4', 'dayofweek_5', 'dayofweek_6',
       'color_green', 'color_red', 'color_yellow', 'hum', 'CO2', 'VOC', 'vis',
       'IR', 'BLE', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'wpgt',
       'pres', 'tmp_diff', 'hum_di

In [7]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(fe.X_train, fe.y_train)

LinearRegression()

In [31]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

pred_test = lr.predict(fe.X_test)

pred_train = lr.predict(fe.X_train)

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_train, y_pred = pred_train)}")

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_test, y_pred = pred_test)}")

Training score: 0.04391886459895278
Training score: 382671425.6814438


In [32]:
fe.X_train.shape

(5680, 71)

In [12]:
time_steps = 7
def create_ds(data, t_steps):
    data = pd.DataFrame(data)
    data_s = data.copy()
    for i in range(time_steps):
        data = pd.concat([data, data_s.shift(-(i+1))], axis = 1)   
    data.dropna(axis=0, inplace=True)
    return data.values

X_train_stepped = create_ds(fe.X_train, 7)

In [13]:
y_train_stepped = create_ds(fe.y_train, 7)

In [156]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout, Bidirectional
from keras.optimizers import Adam, SGD

class MyLSTMModel:
    def __init__(self, lstm_units=int(fe.X_train.shape[1]*2),input_shape=(None, fe.X_train.shape[1])):
        self.model = Sequential()
        self.model.add(LSTM(units=lstm_units, input_shape=input_shape, return_sequences = True))
        self.model.add(Dropout(0.4))
        self.model.add(LSTM(units=lstm_units*2, return_sequences = True))
        self.model.add(Dropout(0.3))
        self.model.add(Dropout(0.3))
        self.model.add(Bidirectional(LSTM(units=lstm_units)))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(50, activation = "relu"))
        self.model.add(Dense(30, activation = "relu"))
        self.model.add(Dense(15, activation = "relu"))

        self.model.add(Dense(1)) 

        self.model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    
    def summary(self):
        self.model.summary()

# Instantiate the model
my_model = MyLSTMModel()

# Print the model summary
X_train_array = fe.X_train.values.reshape(fe.X_train.shape[0], 1, fe.X_train.shape[1]).astype(np.float32)
y_train_array = fe.y_train.values.astype(np.float32).reshape(fe.y_train.shape[0], 1, 1)
my_model.model.fit(X_train_array, y_train_array, epochs = 10)

c:\Users\danie\anaconda3\envs\awp1\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



NameError: name 'keras' is not defined

In [152]:
X_test_array = fe.X_test.values.reshape(fe.X_test.shape[0], 1, fe.X_test.shape[1]).astype(np.float32)
y_test_array = fe.y_test.values.astype(np.float32).reshape(fe.y_test.shape[0], 1, 1)
pred = my_model.model.predict(X_test_array)

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step 


In [153]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

def evaluate_model(y_true, y_pred):
    eval_df = dict()

    eval_df["r2"] = r2_score(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mape"] = mean_absolute_percentage_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mae"] = mean_absolute_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mse"] = mean_squared_error(y_pred = y_pred, y_true = y_true).round(2)

    return pd.DataFrame([eval_df])

evaluate_model(fe.y_test, pred)


,r2,mape,mae,mse
0,-0.48,0.06,1.68,3.86


In [154]:
evaluate_model(fe.y_train, my_model.model.predict(X_train_array))

178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step


,r2,mape,mae,mse
0,0.84,0.04,0.8,0.99


In [155]:
import plotly.express as px

px.line(x = fe.y_test.index, y = [fe.y_test, pd.DataFrame(pred)[0]])